# Plonk & Other IOP Implementation
In this section we will write a Plonk Prover and Verifier, based on a KZG Polynomial Commitment scheme.

In [5]:
# We use BLS 12-381 for the pairing
load("bls-pairing.sage")
# Setup polynomials inside of the G1F
R.<t> = G1F['t'] 

# Set seed for reproducibility
import random

random.seed(int(0))

In [6]:
# Random Oracle represented by SHA256 hash function
from hashlib import sha256
def hash(m):
    hex_hash = sha256(str(m).encode("utf-8")).hexdigest()
    return int(hex_hash, 16)

In [7]:
# KZG (KATE) commitment scheme
class KateCommitmentScheme:
    def __init__(self, max_degree = 2**10):
        # This is the setup for the Kate Commitment Scheme
        self.MAX_DEGREE = max_degree
        self.MAX_BATCH_EVAL = 100

        toxic_waste = G1F.random_element()
        self.secret_powers_in_g1 = [G1 * (toxic_waste ^ x) for x in range (0, self.MAX_DEGREE)]
        self.secret_powers_in_g2 = [G2 * (toxic_waste ^ x) for x in range (0, self.MAX_BATCH_EVAL)]
        del toxic_waste # For explicitness

    # Evaluate the polynomial at a secret point using the notion of the secret cast to elliptic curve point
    def _evaluate_polynomial_at_secret_point(self, polynomial):
        assert  polynomial.degree() < self.MAX_DEGREE, "CRS size is not enough to compute polynomial"
        
        if polynomial.degree() == -1: # This is a 0 polynomial
            return G1 * 0

        # Get a list of coeficients
        polynomial_coef = polynomial.list()
        # Evaluate polynomial at each degree by multiplying the coeficient with the elliptic curve secret to the degree power
        single_degree_eval = [self.secret_powers_in_g1[degree] * polynomial_coef[degree] for degree in range(0, polynomial.degree() + 1)]
        # Sum together all degree to get the final evaluation of the polynomial
        poly_eval = sum(single_degree_eval)
        return poly_eval

    def commit(self, polynomial):
        # Commit by submitting evaluation of a polynomial 
        commitment = self._evaluate_polynomial_at_secret_point(polynomial)
        return commitment
    
    
    def eval_with_proof(self, polynomial, evaluation_point):
        # Evaluate the polynomial at the point
        evaluation = polynomial(evaluation_point)
        # Generate polynomial, that we will use for proof
        proof_polynomial = (polynomial - evaluation) / (t - evaluation_point)
        assert proof_polynomial.is_integral(), "Polynomials must devide each other."
        proof_polynomial = proof_polynomial.numerator()
        # Generate proof by evaluating on toxic point using Reference String
        evaluation_proof = self._evaluate_polynomial_at_secret_point(proof_polynomial)
        return  evaluation, evaluation_proof 
    
    def verify(self, evaluation_point, evaluation, evaluation_proof, commitment): 
        # Compute pairings 
        lhs = BLSpairing.ate_pair(evaluation_proof, (self.secret_powers_in_g2[1] - G2 * evaluation_point))
        rhs = BLSpairing.ate_pair(commitment - G1 * evaluation, G2)
        # Check that two pairings match 
        return lhs == rhs

In [8]:
# Test that KZG Commitment Scheme Works
polynomial = t^1 * 7; evaluation_point = 5
commitment_scheme = KateCommitmentScheme()
commitment = commitment_scheme.commit(polynomial)
(evaluation, proof) = commitment_scheme.eval_with_proof(polynomial, evaluation_point)
assert commitment_scheme.verify(evaluation_point, evaluation, proof, commitment)
"KZG Works!"

'KZG Works!'

### Zero Test IOP Implementation
This test proves that the prove has a polynomial which is 0 on all elements in Omega. Note that if |Omega| > degree(polynomial), then the polynomial must be a zero polynomial. For this case we do not 

In order to make the protocol non-interactive, as the verifier is only doing public coin tosses, we have generated random challenge using hashing of both commitments. This follows Fiat-Shamir Huristic.

In [9]:
class ZeroTestIOP:
    # polynomial - polynomial we want to prove zero test for
    # zero_set - elements of Omega, for which polynomial evaluates to zero
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def prove(polynomial, zero_set, commitment_scheme):
        f = polynomial
        # Commit Polynomial F
        f_commitment = commitment_scheme.commit(f)
        # Eval Q = F / Z
        z = reduce(lambda x, y: x * y, [(t - x) for x in zero_set])
        q = (f / z) if f.degree() > z.degree() else R(1)
        assert q.is_integral(), "Polynomials must devide each other"
        q = q.numerator()
        # Commit Polynomial Q
        q_commitment = commitment_scheme.commit(q)
        
        # In the interactive version, user would have sent commitments to F and Q
        # And would have waited to recieve the challenge point R.
        # However, Fiat-Shamir huristic allows us to instead 
        # just hash the commitments to get the challenge point R.
        
        r = hash((f_commitment, q_commitment))
        
        (f_r, f_r_eval_proof) = commitment_scheme.eval_with_proof(f, r)
        (q_r, q_r_eval_proof) = commitment_scheme.eval_with_proof(q, r)
        
        return (f_commitment, f_r_eval_proof, f_r), (q_commitment, q_r_eval_proof, q_r)
    
    # zero_set - elements of Omega, for which polynomial evaluates to zero
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def verify(zero_set, commitment_scheme, proof):
        f_full_eval_proof, q_full_eval_proof = proof
        (f_commitment, f_r_eval_proof, f_r) = f_full_eval_proof
        (q_commitment, q_r_eval_proof, q_r) = q_full_eval_proof
        # Regenerate r
        r = hash((f_commitment, q_commitment))
        # Check evaluation correctness
        assert commitment_scheme.verify(r, f_r, f_r_eval_proof, f_commitment), "F hash been incorrectly evaluated or committed"
        assert commitment_scheme.verify(r, q_r, q_r_eval_proof, q_commitment), "Q hash been incorrectly evaluated or committed"
        # Check that Q(r) * Z(r) == F(r)
        z = reduce(lambda x, y: x * y, [(t - x) for x in zero_set])
        z_r = z(r)
        assert q_r * z_r == f_r, "Failed to prove that F is zero on all elements in zero set Omega" 

In [10]:
polynomial = (t^2 - 4*t + 3) * (t - 4)
zero_set = [1]
commitment_scheme = KateCommitmentScheme(polynomial.degree() + 1)
proof = ZeroTestIOP.prove(polynomial, zero_set, commitment_scheme)
ZeroTestIOP.verify(zero_set, commitment_scheme, proof)
"ZERO Test Successful"

'ZERO Test Successful'

### Product Check IOP Implementation
This check proves that product of evaulation of the polynomial on all elements of set of roots of unity Omega is equal to 1. This IOP could use the Zero Test IOP in order to prove some subsidiary facts, however to optimise we have chosen not to.

In order to make the protocol non-interactive, as the verifier is only doing public coin tosses, we have generated random challenge using hashing of both commitments. This follows Fiat-Shamir Huristic.

In [11]:
# Function that generates a Lagrange polynomial based on given N evaluations for given N points
# Lagrange polynomial is a polynomial of degree N that passes through all the specified points
def lagrange_poly(points, evaluations):
    
    sum_poly = 0
    for i in range(len(points)):
        
        # This evaluates to 1 if we evaluate polynomial at points[i]
        # At all other specified points it evaluates to 0
        sum_part = 1
        for j in range(len(points)):
            
            # So that we do not devide by zero
            if j == i:
                continue 
            
            # If we evaluate at any point in specified points
            # that is not points[i], we will get one of the product numerators
            # as points[j] - points[j] = 0, thus all product is 0
            product_part = (t - points[j]) / (points[i] - points[j])
            sum_part *= product_part
        
        # This equals to evaluations[i] when evaluate at points[i]
        sum_poly += (evaluations[i] * sum_part)
        
    return sum_poly

In [170]:
# Returns a new polynomial, such that new_poly=poly(x*z)
def mul_poly_eval_point_by(polynomial, z):
    # If the polynomial is fractional, then repeat the function with numerator and denumerator
    if type(polynomial) is sage.rings.fraction_field_element.FractionFieldElement_1poly_field: 
        return mul_poly_eval_point_by(polynomial.numerator(), z) / mul_poly_eval_point_by(polynomial.denominator(), z)
    return sum([(z * t) ^ i_elem[0] * i_elem[1]  for i_elem in enumerate(polynomial)])

In [193]:
class ProductCheckIOP:
    # polynomial - polynomial we want to prove product check for equals to 1
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def prove(polynomial, roots_of_unity, commitment_scheme, do_f_commit = True):
        f = polynomial
        w = roots_of_unity[1]
        k = len(roots_of_unity)
        
        # Commit f only if requried. This is to support quotient polynomials
        f_commit = commitment_scheme.commit(f) if do_f_commit else None
        
        # Compute t by interpolation of its evaluation
        # We have used t_ to denote t to avoid collisions with 't' polynomial definition
        t_evaulations = [f(1)]
        for i in range(1, k):
            t_evaulations += [t_evaulations[-1] * f(roots_of_unity[i])]
        t_ = lagrange_poly(roots_of_unity, t_evaulations) 
        
        # Compute t' that should be equal to 0 everywhere on roots_of_unity
        t_by_w = mul_poly_eval_point_by(t_, w)
        f_by_w = mul_poly_eval_point_by(f, w)
        t_prime = t_by_w - t_ * f_by_w
        
                
        # Eval q = t' / z
        z = t ^ k - 1
        
        q = (t_prime / z) if t_prime.degree() > z.degree() else R(1)
        assert q.is_integral(), "Polynomials must devide each other"
        q = q.numerator()
        
        # Commit to q and t
        q_commit = commitment_scheme.commit(q)
        t_commit = commitment_scheme.commit(t_)
        # In the interactive version, user would have sent commitments to f, t' and q
        # And would have waited to recieve the challenge point r.
        # However, Fiat-Shamir huristic allows us to instead 
        # just hash the commitments to get the challenge point r.
        r = hash((f_commit, q_commit, t_commit))
        
        # Evaluate t at r, wr, w^(k-1)
        (t_r, t_r_eval_proof) = commitment_scheme.eval_with_proof(t_, r)
        commitment_scheme.verify(r, t_r, t_r_eval_proof, t_commit)
        (t_wr, t_wr_eval_proof) = commitment_scheme.eval_with_proof(t_, r * w)
        (t_w_pow_k_1, t_w_pow_k_1_eval_proof) = commitment_scheme.eval_with_proof(t_, w^(k-1))
        
        # Evaluate q at r
        (q_r, q_r_eval_proof) = commitment_scheme.eval_with_proof(q, r)
        
        # Evaluate f at wr
        (f_wr, f_wr_eval_proof) = commitment_scheme.eval_with_proof(f, r * w)
        
        return (f_commit, ((q_commit, t_commit), ((t_r, t_r_eval_proof), (t_wr, t_wr_eval_proof), (t_w_pow_k_1, t_w_pow_k_1_eval_proof), (q_r, q_r_eval_proof), (f_wr, f_wr_eval_proof))))

    # Verify the ProductCheckIOP and return False if it is not correct, otherwise True
    # proof - commitments to f, t, q polynomials as well as  evaluation with proof based on commitments 
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def verify(f_commit, proof, roots_of_unity, commitment_scheme):
        (commitments, evaluations_with_proof) = proof
        (q_commit, t_commit) = commitments
        (t_r, t_r_eval_proof), (t_wr, t_wr_eval_proof), (t_w_pow_k_1, t_w_pow_k_1_eval_proof), (q_r, q_r_eval_proof), (f_wr, f_wr_eval_proof) = evaluations_with_proof
        # Regenerate r
        r = hash((f_commit, q_commit, t_commit))
        # Extract useful info (this is all we actually need to pass to the verifier protocol)
        w = roots_of_unity[1]
        k = len(roots_of_unity)
        
        # Acc is used to see if any of the checks did not pass
        acc = True
        
        # Check that all evaluations were done correctly

        acc &= commitment_scheme.verify(r, t_r, t_r_eval_proof, t_commit)
        acc &= commitment_scheme.verify(r * w, t_wr, t_wr_eval_proof, t_commit)
        acc &= commitment_scheme.verify(w^(k-1), t_w_pow_k_1, t_w_pow_k_1_eval_proof, t_commit)

        
        acc &= commitment_scheme.verify(r, q_r, q_r_eval_proof, q_commit)
        acc &= commitment_scheme.verify(r * w, f_wr, f_wr_eval_proof, f_commit)
        
        # Check that the t' is indeed a zero polynomial over roots_of_unity
        # As well as check that t' = t(wx)-t(x)*f(wx) for roots_of_unity
        # This can be done by combining everything into one equation
        
        z = t^k - 1
        z_r = z(r)
        
        # If f was quatient, then do additional check

        acc &= t_wr - t_r * f_wr == q_r * z_r        
        
        # Check that the product is indeed 1 by checking that t(w^(k-1)) == 1
        
        acc &= t_w_pow_k_1 == 1
        
        
        # All check have passed, if we got here, return True 
        return acc 

In [194]:
# Setup parameters
ROOT_SIZE = 4

# We use w_ instead of w as w is used in bls-pairing file
w_ = G1F.zeta(ROOT_SIZE)
# Note that 0th element is 1 in list of roots_of_unity
roots_of_unity = [w_^i for i in range(ROOT_SIZE)]

# Generate the polynomial that we want to test
polynomial_part_one = lagrange_poly([1, w_, w_^2, w_^3], [1, 4, ~G1F(2), ~G1F(2)]) ^ 2
polynomial_part_two = lagrange_poly([1, w_, w_^2, w_^3], [4, 2, ~G1F(2), ~G1F(4)]) ^ 2 
polynomial = polynomial_part_one * polynomial_part_two 

In [195]:
# Check that the product is indeed 1
poly_product = reduce(lambda x, y: x * y, [polynomial(w_i) for w_i in roots_of_unity])
assert poly_product == 1, "Product of all evaluations over roots_of_unity of the polynomial must be equal to 1"


commitment_scheme = KateCommitmentScheme(polynomial.degree() + 1)
(poly_commit, proof) = ProductCheckIOP.prove(polynomial, roots_of_unity, commitment_scheme)
assert ProductCheckIOP.verify(poly_commit, proof, roots_of_unity, commitment_scheme)
"Product Check Verification Passed"

'Product Check Verification Passed'

### Sum Check IOP Implementation
This check proves that sum of evaulation of the polynomial on all elements of set of roots of unity Omega is equal to 0. This IOP could use the Zero Test IOP in order to prove some subsidiary facts, however to optimise we have chosen not to.

In order to make the protocol non-interactive, as the verifier is only doing public coin tosses, we have generated random challenge using hashing of both commitments. This follows Fiat-Shamir Huristic.

In [174]:
class SumCheckIOP:
    # polynomial - polynomial we want to prove sum check for equals to 0
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def prove(polynomial, roots_of_unity, commitment_scheme, do_f_commit = True):
        f = polynomial
        w = roots_of_unity[1]
        k = len(roots_of_unity)
        
        # Commit f only if requried. This is to support quotient polynomials
        f_commit = commitment_scheme.commit(f) if do_f_commit else None
        
        # Compute t by interpolation of its evaluation
        # We have used t_ to denote t to avoid collisions with 't' polynomial definition
        t_evaulations = [f(1)]
        for i in range(1, k):
            t_evaulations += [t_evaulations[-1] + f(roots_of_unity[i])]
        t_ = lagrange_poly(roots_of_unity, t_evaulations) 
        
        # Compute t' that should be equal to 0 everywhere on roots_of_unity
        t_by_w = mul_poly_eval_point_by(t_, w)
        f_by_w = mul_poly_eval_point_by(f, w)
        t_prime = t_by_w - (t_ + f_by_w)
                
        # Eval q = t' / z
        z = t ^ k - 1
        q = (t_prime / z) if t_prime.degree() > z.degree() else R(1)
        assert q.is_integral(), "Polynomials must devide each other"
        q = q.numerator()
        
        # Commit to q and t
        q_commit = commitment_scheme.commit(q)
        t_commit = commitment_scheme.commit(t_)
        # In the interactive version, user would have sent commitments to f, t' and q
        # And would have waited to recieve the challenge point r.
        # However, Fiat-Shamir huristic allows us to instead 
        # just hash the commitments to get the challenge point r.
        r = hash((f_commit, q_commit, t_commit))
        
        # Evaluate t at r, wr, w^(k-1)
        (t_r, t_r_eval_proof) = commitment_scheme.eval_with_proof(t_, r)
        commitment_scheme.verify(r, t_r, t_r_eval_proof, t_commit)
        (t_wr, t_wr_eval_proof) = commitment_scheme.eval_with_proof(t_, r * w)
        (t_w_pow_k_1, t_w_pow_k_1_eval_proof) = commitment_scheme.eval_with_proof(t_, w^(k-1))
        
        # Evaluate q at r
        (q_r, q_r_eval_proof) = commitment_scheme.eval_with_proof(q, r)
        
        # Evaluate f at wr
        (f_wr, f_wr_eval_proof) = commitment_scheme.eval_with_proof(f, r * w)
        
        return (f_commit, ((q_commit, t_commit), ((t_r, t_r_eval_proof), (t_wr, t_wr_eval_proof), (t_w_pow_k_1, t_w_pow_k_1_eval_proof), (q_r, q_r_eval_proof), (f_wr, f_wr_eval_proof))))

    # Verify the ProductCheckIOP and return False if it is not correct, otherwise True
    # proof - commitments to f, t, q polynomials as well as  evaluation with proof based on commitments 
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def verify(f_commit, proof, roots_of_unity, commitment_scheme):
        (commitments, evaluations_with_proof) = proof
        (q_commit, t_commit) = commitments
        (t_r, t_r_eval_proof), (t_wr, t_wr_eval_proof), (t_w_pow_k_1, t_w_pow_k_1_eval_proof), (q_r, q_r_eval_proof), (f_wr, f_wr_eval_proof) = evaluations_with_proof
        # Regenerate r
        r = hash((f_commit, q_commit, t_commit))
        # Extract useful info (this is all we actually need to pass to the verifier protocol)
        w = roots_of_unity[1]
        k = len(roots_of_unity)
        
        # Acc is used to see if any of the checks did not pass
        acc = True
        
        # Check that all evaluations were done correctly

        acc &= commitment_scheme.verify(r, t_r, t_r_eval_proof, t_commit)
        acc &= commitment_scheme.verify(r * w, t_wr, t_wr_eval_proof, t_commit)
        acc &= commitment_scheme.verify(w^(k-1), t_w_pow_k_1, t_w_pow_k_1_eval_proof, t_commit)

        
        acc &= commitment_scheme.verify(r, q_r, q_r_eval_proof, q_commit)
        acc &= commitment_scheme.verify(r * w, f_wr, f_wr_eval_proof, f_commit)
        
        # Check that the t' is indeed a zero polynomial over roots_of_unity
        # As well as check that t' = t(wx)-t(x)*f(wx) for roots_of_unity
        # This can be done by combining everything into one equation
        
        z = t^k - 1
        z_r = z(r)
        
        acc &= t_wr - (t_r + f_wr) == q_r * z_r        
        
        # Check that the product is indeed 1 by checking that t(w^(k-1)) == 1
        
        acc &= t_w_pow_k_1 == 0
        
        
        # All check have passed, if we got here, return True 
        return acc 

In [175]:
# Setup parameters
ROOT_SIZE = 4

# We use w_ instead of w as w is used in bls-pairing file
w_ = G1F.zeta(ROOT_SIZE)
# Note that 0th element is 1 in list of roots_of_unity
roots_of_unity = [w_^i for i in range(ROOT_SIZE)]

# Generate the polynomial that we want to test
polynomial_part_one = lagrange_poly([1, w_, w_^2, w_^3], [0, -4, 2, 2])
polynomial_part_two = lagrange_poly([1, w_, w_^2, w_^3], [4, 2, 3, 1])
polynomial = polynomial_part_one * polynomial_part_two 

In [176]:
# Check that the sum is indeed 0
poly_sum = reduce(lambda x, y: x + y, [polynomial(w_i) for w_i in roots_of_unity])
assert poly_sum == 0, "Sum of all evaluations over roots_of_unity of the polynomial must be equal to 0"


commitment_scheme = KateCommitmentScheme(polynomial.degree() + 1)
(poly_commit, proof) = SumCheckIOP.prove(polynomial, roots_of_unity, commitment_scheme)
assert SumCheckIOP.verify(poly_commit, proof, roots_of_unity, commitment_scheme)
"Sum Check Verification Passed"

'Sum Check Verification Passed'

### Permutation Check IOP Implementation
This check proves that the group of evaluations of `f` over a set of roots of unity Omega is a permutation of the permutation of group of evaluations of `g` over a set of roots of unity Omega. This IOP makes use of Prouct Check IOP to achive its goal.

In order to make the protocol non-interactive, as the verifier is only doing public coin tosses, we have generated random challenge using hashing of both commitments. This follows Fiat-Shamir Huristic.

In [226]:
# For the PermutationCheckIOP we need a ProductCheckIOP that works on quatient polynomials
class QuatientProductCheckIOP:
    # polynomial - polynomial we want to prove product check for equals to 1
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def prove(polynomial, roots_of_unity, commitment_scheme):
        f = polynomial
        f_num = polynomial.numerator()
        f_denum = polynomial.denominator()
        
        w = roots_of_unity[1]
        k = len(roots_of_unity)
        
        # Commit f by commiting to its numerator and denominator
        f_num_commit = commitment_scheme.commit(f_num)
        f_denum_commit = commitment_scheme.commit(f_denum)
        
        # Compute t by interpolation of its evaluation
        # We have used t_ to denote t to avoid collisions with 't' polynomial definition
        t_evaulations = [f(1)]
        for i in range(1, k):
            t_evaulations += [t_evaulations[-1] * f(roots_of_unity[i])]
        t_ = lagrange_poly(roots_of_unity, t_evaulations) 
        
        # Compute t' that should be equal to 0 everywhere on roots_of_unity
        t_by_w = mul_poly_eval_point_by(t_, w)
        f_by_w = mul_poly_eval_point_by(f, w)
        t_prime = t_by_w - t_ * f_by_w
        
                
        # Eval q = t' / z
        z = t ^ k - 1
        
        t_prime_num = t_prime.numerator()
        t_prime_denum = t_prime.denominator()
        
        t_prime_denum_commit = commitment_scheme.commit(t_prime_denum)

        q = (t_prime_num / z) if t_prime_num.degree() > z.degree() else R(1)
        assert q.is_integral(), "Polynomials must devide each other"
        q = q.numerator()
        
        # Commit to q and t
        q_commit = commitment_scheme.commit(q)
        t_commit = commitment_scheme.commit(t_)
        # In the interactive version, user would have sent commitments to f, t' and q
        # And would have waited to recieve the challenge point r.
        # However, Fiat-Shamir huristic allows us to instead 
        # just hash the commitments to get the challenge point r.
        r = hash((f_num_commit, f_denum_commit, q_commit, t_commit))
        
        # Evaluate t at r, wr, w^(k-1)
        (t_r, t_r_eval_proof) = commitment_scheme.eval_with_proof(t_, r)
        commitment_scheme.verify(r, t_r, t_r_eval_proof, t_commit)
        (t_wr, t_wr_eval_proof) = commitment_scheme.eval_with_proof(t_, r * w)
        (t_w_pow_k_1, t_w_pow_k_1_eval_proof) = commitment_scheme.eval_with_proof(t_, w^(k-1))
        
        # Evaluate t' at r
        t_prime_denum_r, t_prime_denum_r_proof = commitment_scheme.eval_with_proof(t_prime_denum, r)
        
        # Evaluate q at r
        (q_r, q_r_eval_proof) = commitment_scheme.eval_with_proof(q, r)
        
        # Evaluate f at wr
        (f_denum_wr, f_denum_wr_eval_proof) = commitment_scheme.eval_with_proof(f_denum, r * w)
        (f_num_wr, f_num_wr_eval_proof) = commitment_scheme.eval_with_proof(f_num, r * w)
        
        return ((f_num_commit, f_denum_commit), ((q_commit, t_commit, t_prime_denum_commit), ((t_r, t_r_eval_proof), (t_wr, t_wr_eval_proof), (t_w_pow_k_1, t_w_pow_k_1_eval_proof), (t_prime_denum_r, t_prime_denum_r_proof), (q_r, q_r_eval_proof), (f_denum_wr, f_denum_wr_eval_proof), (f_num_wr, f_num_wr_eval_proof))))

    # Verify the ProductCheckIOP and return False if it is not correct, otherwise True
    # proof - commitments to f, t, q polynomials as well as  evaluation with proof based on commitments 
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def verify(f_commit, proof, roots_of_unity, commitment_scheme):
        (f_num_commit, f_denum_commit) = f_commit
        (commitments, evaluations_with_proof) = proof
        (q_commit, t_commit, t_prime_denum_commit) = commitments
        (t_r, t_r_eval_proof), (t_wr, t_wr_eval_proof), (t_w_pow_k_1, t_w_pow_k_1_eval_proof), (t_prime_denum_r, t_prime_denum_r_proof), (q_r, q_r_eval_proof), (f_denum_wr, f_denum_wr_eval_proof), (f_num_wr, f_num_wr_eval_proof) = evaluations_with_proof
        
        # Regenerate r
        r = hash((f_num_commit, f_denum_commit, q_commit, t_commit))
        # Extract useful info (this is all we actually need to pass to the verifier protocol)
        w = roots_of_unity[1]
        k = len(roots_of_unity)
        
        # Acc is used to see if any of the checks did not pass
        acc = True
        
        # Check that all evaluations were done correctly

        acc &= commitment_scheme.verify(r, t_r, t_r_eval_proof, t_commit)
        acc &= commitment_scheme.verify(r * w, t_wr, t_wr_eval_proof, t_commit)
        acc &= commitment_scheme.verify(w^(k-1), t_w_pow_k_1, t_w_pow_k_1_eval_proof, t_commit)

        
        acc &= commitment_scheme.verify(r, q_r, q_r_eval_proof, q_commit)
        acc &= commitment_scheme.verify(r * w, f_num_wr, f_num_wr_eval_proof, f_num_commit)
        acc &= commitment_scheme.verify(r * w, f_denum_wr, f_denum_wr_eval_proof, f_denum_commit)

        
        # Check that the t' is indeed a zero polynomial over roots_of_unity
        # As well as check that t' = t(wx)-t(x)*f(wx) for roots_of_unity
        # This can be done by combining everything into one equation
        
        z = t^k - 1
        z_r = z(r)
        
        # If f was quatient, then do additional check
        acc &= commitment_scheme.verify(r, t_prime_denum_r, t_prime_denum_r_proof, t_prime_denum_commit)
        acc &= t_wr - t_r * (f_num_wr / f_denum_wr) == q_r * z_r / t_prime_denum_r       
        
        # Check that the product is indeed 1 by checking that t(w^(k-1)) == 1
        
        acc &= t_w_pow_k_1 == 1
        
        
        # All check have passed, if we got here, return True 
        return acc 

In [237]:
class PermutationCheckIOP:
    # polynomial_f - polynomial we want to prove the permutation check for
    # polynomial_g - polynomial we want to prove the permutation check for
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def prove(polynomial_f, polynomial_g, roots_of_unity, commitment_scheme):
        
        f = polynomial_f
        g = polynomial_g
        
        # Commit f and g
        f_commit = commitment_scheme.commit(f)
        g_commit = commitment_scheme.commit(g)
        
        # Generate challenge point r using Fiat Shamir
        r = hash((f_commit, g_commit))
    
        # Generate evaluation of f and g at r
        (f_r, f_r_proof) = commitment_scheme.eval_with_proof(f, r)
        (g_r, g_r_proof) = commitment_scheme.eval_with_proof(g, r)

        # Product check poly and its evaluation at r, as we can not commit fractions, we commit subparts
        numer_poly = r - f
        denum_poly = r - g
        prod_poly = numer_poly / denum_poly

        num_poly_commit = commitment_scheme.commit(numer_poly)
        denum_poly_commit = commitment_scheme.commit(denum_poly)
        
        (num_poly_r, num_poly_r_proof) = commitment_scheme.eval_with_proof(numer_poly, r)
        (denum_poly_r, denum_poly_r_proof) = commitment_scheme.eval_with_proof(denum_poly, r)
        
        prod_commit, prod_proof = QuatientProductCheckIOP.prove(prod_poly, roots_of_unity, commitment_scheme)
        
        return (f_commit, g_commit, ((num_poly_commit, denum_poly_commit), (prod_commit, prod_proof), (num_poly_r, num_poly_r_proof), (denum_poly_r, denum_poly_r_proof), (f_r, f_r_proof), (g_r, g_r_proof)))
        
    
    # Verify the ProductCheckIOP and return False if it is not correct, otherwise True
    # proof - commitments to f, t, q polynomials as well as  evaluation with proof based on commitments 
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def verify(f_commit, g_commit, proof, roots_of_unity, commitment_scheme):
        
        # Deconstruct the proof
        ((num_poly_commit, denum_poly_commit), (prod_commit, prod_proof), (num_poly_r, num_poly_r_proof), (denum_poly_r, denum_poly_r_proof), (f_r, f_r_proof), (g_r, g_r_proof)) = proof

        acc = True
        
        # Do the product check
        acc &= QuatientProductCheckIOP.verify(prod_commit, prod_proof, roots_of_unity, commitment_scheme)
                
        # Regenerate challenge point r
        r = hash((f_commit, g_commit))
        
        # Check evaluations of f and g
        acc &= commitment_scheme.verify(r, f_r, f_r_proof, f_commit)
        acc &= commitment_scheme.verify(r, g_r, g_r_proof, g_commit)
        
        # Check evaluations of prod_check_poly
        acc &= commitment_scheme.verify(r, num_poly_r, num_poly_r_proof, num_poly_commit)
        acc &= commitment_scheme.verify(r, denum_poly_r, denum_poly_r_proof, denum_poly_commit)
        
        # Now we need to check that the prod_check_poly has been constructed correctly from f and g at r
        acc &= (r - f_r) / (r - g_r) == num_poly_r / denum_poly_r
        
        
        return acc

In [238]:
# Setup parameters
ROOT_SIZE = 4

# We use w_ instead of w as w is used in bls-pairing file
w_ = G1F.zeta(ROOT_SIZE)
# Note that 0th element is 1 in list of roots_of_unity
roots_of_unity = [w_^i for i in range(ROOT_SIZE)]

# Generate the polynomials using Lagrange 
polynomial_f = lagrange_poly([1, w_, w_^2, w_^3], [0, -4, 2, 2])
polynomial_g = lagrange_poly([1, w_, w_^2, w_^3], [2, -4, 2, 0])

# Verify that polynomials follow the permutation check
polynomial_f_eval = set([polynomial_f(w_i) for w_i in roots_of_unity])
polynomial_g_eval = set([polynomial_g(w_i) for w_i in roots_of_unity])
assert polynomial_f_eval == polynomial_g_eval, "Polynomial do not abey permutation check"

# Run PermutationCheckIOP 
commitment_scheme = KateCommitmentScheme(max(polynomial_f.degree(), polynomial_g.degree()) + 1)
f_commit, g_commit, proof = PermutationCheckIOP.prove(polynomial_f, polynomial_g, roots_of_unity, commitment_scheme)
assert PermutationCheckIOP.verify(f_commit, g_commit, proof, roots_of_unity, commitment_scheme), "Permutation Check must hold"
'Permutation Check Verification Passed'

'Permutation Check Verification Passed'

### Prescribed Permutation Check IOP Implementation
This check proves that `f(y)` equals to `g(W(x))` over a set of roots of unity Omega and for some permutation W. This IOP makes use of Quatient Prouct Check IOP to achive its goal.

In order to make the protocol non-interactive, as the verifier is only doing public coin tosses, we have generated random challenge using hashing of both commitments. This follows Fiat-Shamir Huristic.

In [241]:
class PrescribedPermutationCheckIOP:
    # polynomial_f - polynomial we want to prove the permutation check for
    # polynomial_g - polynomial we want to prove the permutation check for
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def prove(polynomial_f, polynomial_g, permutation_w, roots_of_unity, commitment_scheme):
        
        f = polynomial_f
        g = polynomial_g
        w = permutation_w
        
        # Commit f and g
        f_commit = commitment_scheme.commit(f)
        g_commit = commitment_scheme.commit(g)
        w_commit = commitment_scheme.commit(w)
        
        # Generate a pair of challenge points r and s using Fiat Shamir
        r, s = hash((f_commit, g_commit, w_commit, 0)), hash((f_commit, g_commit, w_commit, 1))
    
        # Generate evaluation of f, g and w at r
        (f_r, f_r_proof) = commitment_scheme.eval_with_proof(f, r)
        (g_r, g_r_proof) = commitment_scheme.eval_with_proof(g, r)
        (w_r, w_r_proof) = commitment_scheme.eval_with_proof(w, r)

        # Product check poly and its evaluation at r, as we can not commit fractions, we commit subparts
        numer_poly = r - s * w - f
        denum_poly = r - s * t - g
        prod_poly = numer_poly / denum_poly

        num_poly_commit = commitment_scheme.commit(numer_poly)
        denum_poly_commit = commitment_scheme.commit(denum_poly)
        
        (num_poly_r, num_poly_r_proof) = commitment_scheme.eval_with_proof(numer_poly, r)
        (denum_poly_r, denum_poly_r_proof) = commitment_scheme.eval_with_proof(denum_poly, r)
        
        prod_commit, prod_proof = QuatientProductCheckIOP.prove(prod_poly, roots_of_unity, commitment_scheme)
        
        return (f_commit, g_commit, w_commit, ((num_poly_commit, denum_poly_commit), (prod_commit, prod_proof), (num_poly_r, num_poly_r_proof), (denum_poly_r, denum_poly_r_proof), (f_r, f_r_proof), (g_r, g_r_proof), (w_r, w_r_proof)))
        
    
    # Verify the ProductCheckIOP and return False if it is not correct, otherwise True
    # proof - commitments to f, t, q polynomials as well as  evaluation with proof based on commitments 
    # roots_of_unity - w^i for i from 0 to n-1, such that w^n = 1 and
    # commitment_scheme - commitment scheme of choice
    @staticmethod
    def verify(f_commit, g_commit, w_commit, proof, roots_of_unity, commitment_scheme):
        
        # Deconstruct the proof
        ((num_poly_commit, denum_poly_commit), (prod_commit, prod_proof), (num_poly_r, num_poly_r_proof), (denum_poly_r, denum_poly_r_proof), (f_r, f_r_proof), (g_r, g_r_proof), (w_r, w_r_proof)) = proof

        acc = True
        
        # Do the product check
        acc &= QuatientProductCheckIOP.verify(prod_commit, prod_proof, roots_of_unity, commitment_scheme)
                
        # Regenerate a pair of challenge points r and s
        r, s = hash((f_commit, g_commit, w_commit, 0)), hash((f_commit, g_commit, w_commit, 1))
        
        # Check evaluations of f and g
        acc &= commitment_scheme.verify(r, f_r, f_r_proof, f_commit)
        acc &= commitment_scheme.verify(r, g_r, g_r_proof, g_commit)
        acc &= commitment_scheme.verify(r, w_r, w_r_proof, w_commit)
        
        # Check evaluations of prod_check_poly
        acc &= commitment_scheme.verify(r, num_poly_r, num_poly_r_proof, num_poly_commit)
        acc &= commitment_scheme.verify(r, denum_poly_r, denum_poly_r_proof, denum_poly_commit)
        
        # Now we need to check that the prod_check_poly has been constructed correctly from f and g at r
        acc &= (r - s * w_r - f_r) / (r - s * r - g_r) == num_poly_r / denum_poly_r
        
        return acc

In [242]:
# Setup parameters
ROOT_SIZE = 4

# We use w_ instead of w as w is used in bls-pairing file
w_ = G1F.zeta(ROOT_SIZE)
# Note that 0th element is 1 in list of roots_of_unity
roots_of_unity = [w_^i for i in range(ROOT_SIZE)]

# Generate the polynomials using Lagrange 
permutation_w = lagrange_poly([1, w_, w_^2, w_^3], [w_, 1, w_^3, w_^2])
polynomial_f = lagrange_poly([1, w_, w_^2, w_^3], [0, -4, 2, 2])
polynomial_g = lagrange_poly([1, w_, w_^2, w_^3], [-4, 0, 2, 2])

# Verify that polynomials follow the permutation check
for w_ in roots_of_unity:
    assert polynomial_g(permutation_w(w_)) == polynomial_f(w_),  "Polynomial do not abey prescribed permutation check"
    

# Run PrescribedPermutationCheckIOP 
commitment_scheme = KateCommitmentScheme(max(polynomial_f.degree(), polynomial_g.degree()) + 1)
f_commit, g_commit, w_commit, proof = PrescribedPermutationCheckIOP.prove(polynomial_f, polynomial_g, permutation_w, roots_of_unity, commitment_scheme)
assert PrescribedPermutationCheckIOP.verify(f_commit, g_commit, w_commit, proof, roots_of_unity, commitment_scheme), "Permutation Check must hold"
'Prescribed Permutation Check Verification Passed'

'Prescribed Permutation Check Verification Passed'

### Plonk Prover and Verifier 
Using the IOPs above we can now construct a Plonk prover and verifier.

We first define a structure to build our circuits, which we then convert into a Plonk friendly format and prove.

In [322]:
class MulGate:
    # left_elem is either input or another gate
    # right_elem is either input or another gate
    def __init__(self, left_elem, right_elem):
        self.left = left_elem
        self.right = right_elem
        
        self.output_users = set()

    
    def evaluate(self):
        left_eval = self.left.evaluate()
        right_eval = self.right.evaluate()
        return left_eval * right_eval
    
    def evaluate_with_left_right(self):
        left_eval = self.left.evaluate()
        right_eval = self.right.evaluate()
        return [left_eval, right_eval, left_eval * right_eval]
    
    
    # Register index in children to show which get index uses their output
    def register_in_children(my_index):
        self.left.output_users.add(my_index * 3)
        self.right.output_users.add(my_index * 3 + 1)

In [329]:
class AddGate:
    # left_elem is either input or another gate
    # right_elem is either input or another gate
    def __init__(self, left_elem, right_elem):
        self.left = left_elem
        self.right = right_elem
        
        self.output_users = set()
    
    def evaluate(self):
        left_eval = self.left.evaluate()
        right_eval = self.right.evaluate()
        return left_eval + right_eval
    
    def evaluate_with_left_right(self):
        left_eval = self.left.evaluate()
        right_eval = self.right.evaluate()
        return [left_eval, right_eval, left_eval + right_eval]
    
    
    # Register index in children to show which get index uses their output
    def register_in_children(my_index):
        self.left.output_users.add(my_index * 3)
        self.right.output_users.add(my_index * 3 + 1)

In [340]:
class PublicInputGate:
    def __init__():
        self.output_users = set()

    # value to assign to the input gate
    def assign(self, value):
        self.value = value
    
    def evaluate(self):
        return self.value

In [341]:
type(PublicInputGate())

<class '__main__.PublicInputGate'>

In [ ]:
class WitnessInputGate:
    # value to assign to the input gate
    def assign(self, value):
        self.value = value
    
    def evaluate(self):
        return self.value

In [335]:
# Define a circuit as
#           _*_       
#          /   \        
#       _+_   _+_         
#      /   \ /   \             
#     x1    x2   x3

gate1 = InputGate()
gate2 = InputGate()
gate3 = WitnessInputGate()

gate4 = AddGate(gate1, gate2)
gate5 = AddGate(gate2, gate3)

gate6 = MulGate(gate4, gate5)

In [336]:
gate1.assign(5)
gate2.assign(6)
gate3.assign(1)

gate6.evaluate()

77

In [346]:
class Circuit:
    def __init__(self, all_gates, input_size):
        
        self.input_gates = all_gates[:input_size]
        self.output_gate = all_gates[-1]
        self.action_gates = all_gates[input_size:]
                
        
    def evaluate(self, assignments):
        assert len(self.input_gates) == len(assignments), 'Assignment size is incorrect'
        for input_gate, assignment in zip(self.input_gates, assignments):
            input_gate.assign(assignment)
            
        return self.output_gate.evaluate()
    
    def prove(self, assignments):
        
        d = len(self.all_gates) * 3 - 2 * len(self.input_gates)
        
        w = G1F.zeta(d)
        roots_of_unity = [w_^i for i in range(d)]
        
        # This is inefficient as we can reuse previous calculations
        roots_evaluations = []
        for action_gate in self.action_gates:
            roots_evaluations += action_gate.evaluate_with_left_right()
            
        roots_evaluations += assignments
        
        # Build a trace polynomial based on the evaluation of the circuit
        T = lagrange_poly(roots_of_unity, roots_evaluations)
        
        
        # Genrate a commitment to trace polynomial T
        commitment_scheme = KateCommitmentScheme(T.degree() + 1)

        T_commit = commitment_scheme.commit(T)
        T_r, T_r_proof = commitment_scheme.eval_with_proof(T, r)
        T_rw, T_rw_proof = commitment_scheme.eval_with_proof(T, r * w)
        T_rww, T_rww_proof = commitment_scheme.eval_with_proof(T, r * w * w)


        
        # Prove the 4 qualities:
        # 1. Inputs are correctly encoded
        # 2. Every gate is evaluated correctly
        # 3. The wiring is implemented correctly
        # 4. The output of last gate is X
        
        
        # Stage 1: proving inputs x
        
        # We interpolate v such that it evaluates to public input x
        public_inputs = []
        for input_gate, assignment in zip(self.input_gates, assignments):
            if input_gate is __main__.PublicInputGate:
                public_inputs += assignment
                
        omega_inp = roots_of_unity[-len(public_inputs):]
        v = lagrange_poly(omega_inp, public_inputs)
        
        # TODO - check if order of public inputs relative to private matters!
        # This is likely a place for a permutation check
        T_v_commit = commitment_scheme.commit(T - v)
        r = hash(public_inputs) # Should depend on comm S and T_v
        T_v_r, T_v_r_proof = commitment_scheme.eval_with_proof(T - v, r)
        one_zero_test_proof = ZeroTestIOP.prove(T - v, omega_inp, commitment_scheme)
        
        # Stage 2: proving gate evaluations
        
        # s(w^3l) is equal to 0 if gate l is addition, 1 otherwise
        s_evaluations = []
        s_points = []
        for i, gate in enumerate(self.action_gates):
            if gate is __main__.AddGate:
                s_evaluations += [1]
                s_points = [w^(3 * i)]
        
        S = lagrange_poly(s_points, s_evaluations)
        S_commit = commitment_scheme.commit(S)
        S_r, S_r_proof = commitment_scheme.eval_with_proof(S, r)
        
        two_zero_test_poly = S * (T + mul_poly_eval_point_by(T, w)) + (1 - S) * T * mul_poly_eval_point_by(T, w^2)
        two_zero_test_poly_commit = commitment_scheme.commit(two_zero_test_poly)
        two_zero_test_poly_r, two_zero_test_poly_r_proof = commitment_scheme.eval_with_proof(two_zero_test_poly, r)
        two_zero_test_proof = ZeroTestIOP.prove(two_zero_test_poly, s_points, commitment_scheme)
        
        
        # Stage 3: proving wiering of gates
        
        
        for i, action_gate in enumerate(self.action_gates):
            action_gate.register_in_children(i)
            
        permutation_sets = []
        for i, action_gate in enumerate(self.action_gates):
            permutation_sets += [action_gate.output_users + [i * 3 + 2]]
            
        for i, input_gate in enumerate(self.input_gates):
            permutation_sets += [action_gate.output_users + [len(self.input_gates) - i]]
            
        
        permutation_eval_points = []
        permutation_evaluations = []
        for permutation_set in permutation_sets:
            permutation_eval_points += [w^i in permutation_set]
            permutation_evaluations += [w^i in (permutation_set[:-1] + permutation_set[0])]
            
        permutation_w = lagrange_poly(permutation_eval_points, permutation_evaluations)
        
        _, _, w_commit, proof = PrescribedPermutationCheckIOP.prove(T, T, permutation_w, roots_of_unity, commitment_scheme)        
        three_permutation_proof = (w_commit, proof)

        # Stage 4: prove the evaluation of the last gate is equal to X
        
        circuit_eval = self.evaluate(assignments)
        _, four_circuit_eval_proof = commitment_scheme.eval_with_proof(T, w^(3 * len(self.action_gates) - 1))


        return T_commit, one_zero_test_proof, two_zero_test_proof, three_permutation_proof, four_circuit_eval_proof, ((T_r, T_r_proof), (T_rw, T_rw_proof), (T_rww, T_rww_proof), (T_v_commit, T_v_r, T_v_r_proof), (S_commit, S_r, S_r_proof), (two_zero_test_poly_commit, two_zero_test_poly_r, two_zero_test_poly_r_proof))
    
    
    def verify_circuit_evaluation(self, public_inputs, circuit_eval, proof):
        (T_commit, one_zero_test_proof, two_zero_test_proof, three_permutation_proof, four_circuit_eval_proof, T_evaluations) = proof

        ((T_r, T_r_proof), (T_rw, T_rw_proof), (T_rww, T_rww_proof), (T_v_commit, T_v_r, T_v_r_proof), (S_commit, S_r, S_r_proof), (two_zero_test_poly_commit, two_zero_test_poly_r, two_zero_test_poly_r_proof)) = T_evaluations
        d = len(self.all_gates) * 3 - 2 * len(self.input_gates)

        w = G1F.zeta(d)
        roots_of_unity = [w_^i for i in range(d)]
        
        acc = True
        
        acc &= commitment_scheme.verify(r, T_r, T_r_proof, T_commit)
        acc &= commitment_scheme.verify(r, T_rw, T_rw_proof, T_commit)
        acc &= commitment_scheme.verify(r, T_rww, T_rww_proof, T_commit)

        
        # Check stage 1:
        # We interpolate v such that it evaluates to public input x
        public_inputs = []
        for input_gate, assignment in zip(self.input_gates, assignments):
            if input_gate is __main__.PublicInputGate:
                public_inputs += assignment
                
        omega_inp = roots_of_unity[-len(public_inputs):]
        v = lagrange_poly(omega_inp, public_inputs)

        r = hash(public_inputs) # This should be replaced with dependdancy on circuit and public inputs, as shoudl be independant of gate size during runtime 
        acc &= commitment_scheme.verify(r, T_v_r, T_v_r_proof, T_v_commit)
        acc &= T_v_r == T_r - v(r) 
        
        ZeroTestIOP.verify(one_zero_test_proof, omega_inp, commitment_scheme)
        
        # Check stage 2:
        s_points = []
        for i, gate in enumerate(self.action_gates):
            if gate is __main__.AddGate:
                s_evaluations += [1]
                s_points = [w^(3 * i)]
        acc &= ZeroTestIOP.verify(two_zero_test_proof, omega_inp, commitment_scheme)
        acc &= commitment_scheme.verify(r, S_r, S_r_proof, S_commit)
        acc &= commitment_scheme.verify(r, two_zero_test_poly_r, two_zero_test_poly_r_proof, two_zero_test_poly_commit)
        acc &= S_r * (T_r * T_rw) + (1 - S_r) * T_r * T_rw - T_rww == two_zero_test_poly_r
        
        # Check state 3:

        acc &= PrescribedPermutationCheckIOP.verify(T_commit, T_commit, w_commit, three_permutation_proof, roots_of_unity, commitment_scheme)
        
        # Check state 4:
        
        acc &= commitment_scheme.verify(w ^ (3 * len(self.action_gates) - 1), 0, four_circuit_eval_proof, T_commit)
        
        return acc

In [ ]:
#### Warning: The code is still in development and needs to be debugged.
#### In particular, there are alot of insecurities in the code, and the code is not yet optimized.

In [338]:
circuit = Circuit([gate1, gate2, gate3, gate4, gate5, gate6], 3)
evaluation = circuit.evaluate([5, 6, 1])
proof = circuit.prove([5, 6, 1])
assert circuit.verify_circuit_evaluation([5, 6], evaluation, proof) == True, "Plonk proof of circuit evaluation did not pass"

[5, 6, 11, 6, 1, 7, 11, 7, 77, 5, 6, 1]


In [299]:
print(type(gate1))

<class '__main__.InputGate'>


In [343]:
[1, 2][-1:]

[2]